In [ ]:
/************************************************************************
 * Last Run Date: 25 March 2017	            							*
 * Program Name: panders2_hw09_prog.sas									*
 * Program Location: C:/Users/Philip/Schools/TAMU/STAT_657/				*
 *				SASUniversityEdition/myfolders/homework/git_hw/hw_09/sasprogs/	* 
 * Creation Date: 25 March 2017											*
 * Author: Philip Anderson												*
 * Purpose: Meeting objectives for Assignment 08						*
 * Inputs: orion folder         *
 * Output: panders2_hw09_output.pdf															*
 * Modification History: *
 * Execution Instructions: N/A - run as-is												*
*/**********************************************************************;


In [3]:
/* Question 1 - set a system option that writes macro as they are resolved */
/* writes to the SAS log text created generated by macro execution */
options symbolgen mcompilenote=ALL;

/* specify the hw09 libname */
libname sasdata '/folders/myfolders/homework/git_hw/hw_09/sasdata' ;


In [4]:
/* 
Question 2 - copy the proc sql code that was used to create the view in assignment 8 and paste it into this program
I'll be using the code from the solutions
*/

/*assign the libref in the body of the program*/
libname orion '/folders/myfolders/orion_dat/sasdata' access=readonly;

/* assign macro variables for gender, start_date, end_date, and libname for output table */
%let gender = F;
%let enddt = 31Dec2006;
%let startdt = 01Jan2006;
%let outlib = work;


proc sql;
/* change the code so that it creates a table instead of a view */
create table &outlib..femdonors_test as
select 
c.employee_id 'ID'
, employee_name 'Name'
, salary format=dollar8.
, qtr1
, qtr2
, qtr3
, qtr4
, sum(qtr1, qtr2, qtr3, qtr4) as tot_donation 'Ann. Donation'
from 
(select 
a.employee_id
, employee_name
, salary
 from orion.employee_payroll a
 , orion.employee_addresses b
 where a.employee_id=b.employee_id 
 and employee_term_date is missing 
 and (employee_hire_date between "&startdt."d and "&enddt."d) 
 and employee_gender = "&gender.") as c

 left join
 orion.employee_donations as d
 on c.employee_id=d.employee_id
 ;
 quit;

In [5]:
/* Question 3 - Use a proc print to print the data portion of the data set by using the SYSLAST macro variable */
title 'Data Portion of the &SYSLAST Data Set';
proc print data=&SYSLAST.;
run;

Obs,Employee_ID,Employee_Name,Salary,Qtr1,Qtr2,Qtr3,Qtr4,tot_donation
1,120108,"Gromek, Gladys","$27,660",.,.,.,.,.
2,120109,"Baker, Gabriele","$26,495",.,.,.,.,.
3,120128,"Kletschkus, Monica","$30,890",.,.,.,.,.
4,120133,"Soltau, Petrea","$27,440",.,.,.,.,.
5,120137,"Iyengar, Marina","$29,715",.,.,.,.,.
6,120138,"Duckett, Shani","$25,795",.,.,.,.,.
7,120139,"Wilson, Fang","$26,810",.,.,.,.,.
8,120141,"Liebman, Amanda","$27,465",.,.,.,.,.
9,120147,"Rusli, Skev","$26,580",.,.,.,.,.
10,120155,"James, Narelle","$29,990",.,.,.,.,.


In [25]:

%let gender = male;
%let enddt = 30Jun1974;
%let startdt = 01Jan1974;

data test;
/* test_var = substr("&startdt.", 6, 4);
test_two = year("&startdt."d);
gender_var = propcase("&gender.");
concat = compress(gender_var||test_two); */
concat_2 = compress(propcase("&gender.")||year("&startdt."d));
call symputx('dset', concat_2);
run;

%put &dset.;

In [30]:
/* 
Question 4 - copy the SQL section created above.  Create a macro with arguments to run the entire sql procedure in one line 
When calling this macro, specify the fully spelled out Gender name 
*/


%macro donations(outlib, gender, startdt, enddt);

/* 
construct the data set name automatically in the form of GenderSTARTYEAR 
This will help with that
*/
data _null_;
concat_2 = compress(propcase("&gender.")||year("&startdt."d));
call symputx('dset', concat_2);
run;

proc sql;
create table &&outlib..&dset. as 
select 
c.employee_id 'ID'
, employee_name 'Name'
, salary format=dollar8.
, qtr1
, qtr2
, qtr3
, qtr4
, sum(qtr1, qtr2, qtr3, qtr4) as tot_donation 'Ann. Donation'
from 

(select 
a.employee_id
, employee_name
, salary
 from orion.employee_payroll a
 , orion.employee_addresses b
 where a.employee_id=b.employee_id 
 and employee_term_date is missing 
 and (employee_hire_date between "&startdt."d and "&enddt."d) 
 /* parse out the employee gender from the user-function input */
 and employee_gender = substr(propcase("&gender."), 1, 1)) as c

 left join
 orion.employee_donations as d
 on c.employee_id=d.employee_id
 ;
 
 /*print out the results of the query*/
 /*construct the title */
 title "Donations of Male Employees Hired between &startdt. and &enddt.";
 footnote "&&outlib..&SYSLAST.";
 select
 *
 from
 &&outlib..&dset.
 ;
 footnote;
 quit;
 %mend;

In [32]:
/*Question 5 - call the macro wtih specified parameters */
%donations(sasdata, male, 01Jan1974, 30Jun1974);

ID,Name,Salary,Qtr1,Qtr2,Qtr3,Qtr4,Ann. Donation
120103,"Dawes, Wilson","$87,975",.,.,.,.,.
120106,"Hornsey, John","$26,960",.,.,.,.,.
120172,"Comber, Edwin","$28,345",.,.,.,.,.
120178,"Plested, Billy","$26,165",.,.,.,.,.
120270,"Nuss, Grezegorz","$48,435",20,10,5,.,35
120272,"Flow, Febin","$34,390",10,10,10,10,40
120659,"Havasy, Jay","$161,290",.,.,.,.,.
120669,"Hill, Ronald","$36,370",15,15,15,15,60
120691,"Habres, Sek","$49,240",.,.,.,.,.
120713,"Campbell, Carston","$31,630",15,15,15,15,60


In [42]:
/* Question 6
copy the macro definition code and paste it at the bottom of your SAS program 
use the proper options to store it in your permanent library specified in part 1
*/
options mstored sasmstore=sasdata;

%macro donations(outlib, gender, startdt, enddt) / STORE;

/* 
construct the data set name automatically in the form of GenderSTARTYEAR 
This will help with that
*/
data _null_;
concat_2 = compress(propcase("&gender.")||year("&startdt."d));
call symputx('dset', concat_2);
run;

proc sql;
create table &&outlib..&dset. as 
select 
c.employee_id 'ID'
, employee_name 'Name'
, salary format=dollar8.
, qtr1
, qtr2
, qtr3
, qtr4
, sum(qtr1, qtr2, qtr3, qtr4) as tot_donation 'Ann. Donation'
from 

(select 
a.employee_id
, employee_name
, salary
 from orion.employee_payroll a
 , orion.employee_addresses b
 where a.employee_id=b.employee_id 
 and employee_term_date is missing 
 and (employee_hire_date between "&startdt."d and "&enddt."d) 
 /* parse out the employee gender from the user-function input */
 and employee_gender = substr(propcase("&gender."), 1, 1)) as c

 left join
 orion.employee_donations as d
 on c.employee_id=d.employee_id
 ;
 
 /*print out the results of the query*/
 /*construct the title */
 title "Donations of Male Employees Hired between &startdt. and &enddt.";
 footnote "&&outlib..&SYSLAST.";
 select
 *
 from
 &&outlib..&dset.
 ;
 footnote;
 quit;
 %mend;


In [41]:
/* Question 7 - use the catalog procedure to list macros stored in your permanent library*/
proc catalog cat=sasdata.sasmacr;
    contents;
    title "Compiled Macros in My Permanent Library";
run;